In [1]:

import os
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path 
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [2]:
# Path to the data directory
data_dir = Path("./samples")

images = sorted(list(map(str, list(data_dir.glob("*.png")))))
labels = [img.split(os.path.sep)[-1].split(".png")[0] for img in images]
characters = set(char for label in labels for char in label)
characters = sorted(characters)
print(f"Number of images found: {len(images)}")
print(f"Number of labels found: {len(labels)}")
print(f"Number of unique characters: {len(characters)}")
print(f"Characters present: {characters}")

Number of images found: 1024
Number of labels found: 1024
Number of unique characters: 23
Characters present: ['2', '3', '4', '5', '6', '7', '8', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'm', 'n', 'p', 's', 'w', 'x', 'y', 'z']


In [3]:
# Batch size for training and validation
batch_size = 16

# Desired image dimensions
img_width = 300
img_height = 50

# Factor by which the image is going to be downsampled by the convolutional blocks.
# We will be using two convolution blocks and each block will have a pooling layer
# which downsample the features by a factor of 2.
# Hence total downsampling factor would be 4.
downsample_factor = 4

# Maximum length of any captcha in the dataset
max_length = max([len(label) for label in labels])
print(max_length)
for label in labels:
    if(len(label)<6):
        print(label)
     

6


# Preprocessing

In [4]:
# Mapping characters to integers
char_to_num = layers.experimental.preprocessing.StringLookup(
    vocabulary=list(characters), num_oov_indices=0, mask_token=None
)

# Mapping integers back to original characters
num_to_char = layers.experimental.preprocessing.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)


def split_data(images, labels, train_size=0.9, shuffle=True):
    # 1. Get the total size of the dataset
    size = len(images)
    # 2. Make an indices array and shuffle it, if required
    indices = np.arange(size)
    if shuffle:
        np.random.shuffle(indices)
    # 3. Get the size of training samples
    train_samples = int(size * train_size)
    # 4. Split data into training and validation sets
    x_train, y_train = images[indices[:train_samples]], labels[indices[:train_samples]]
    x_valid, y_valid = images[indices[train_samples:]], labels[indices[train_samples:]] 
    return x_train, x_valid, y_train, y_valid

def encode_single_sample(img_path, label):
    # 1. Read image
    img = tf.io.read_file(img_path)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img, [img_height, img_width])
    # 5. Transpose the image because we want the time
    # dimension to correspond to the width of the image.
    img = tf.transpose(img, perm=[1, 0, 2])
    # 6. Map the characters in label to numbers
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    # 7. Return a dict as our model is expecting two inputs
    return {"image": img, "label": label}

/Users/pouria/mambaforge/envs/ten/lib/python3.11/site-packages/numpy/core/numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [5]:
# Splitting data into training and validation sets
X_train, X_valid, y_train, y_valid = split_data(np.array(images), np.array(labels))

print(f"X_Train shape: {X_train.shape}")
print(f"y_Train shape: {y_train.shape}")
print(f"X_valid shape: {X_valid.shape}")
print(f"y_valid shape: {y_valid.shape}")

AttributeError: 'NoneType' object has no attribute 'astype'

# Create dataset objects

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = (
    train_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
)

validation_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
validation_dataset = (
    validation_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
)

# Visualize the data

In [ ]:
_, ax = plt.subplots(4, 4, figsize=(10, 5))
for batch in train_dataset.take(1):
    images = batch["image"]
    labels = batch["label"]
    for i in range(16):
        img = (images[i] * 255).numpy().astype("uint8")
        label = tf.strings.reduce_join(num_to_char(labels[i])).numpy().decode("utf-8")
        ax[i // 4, i % 4].imshow(img[:, :, 0].T, cmap="gray")
        ax[i // 4, i % 4].set_title(label)
        ax[i // 4, i % 4].axis("off")
plt.show()


# Model

In [ ]:

class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost
 
        
    def call(self, y_true, y_pred):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")
        
        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)
        
        # At test, just return the computed predictions
        return y_pred
    
def build_model():
    # Inputs the model
    input_img = layers.Input(
        shape=(img_width, img_height, 1), name="image", dtype="float32"
    )
    labels = layers.Input(name="label", shape=(None,), dtype="float32")
    
    # First conv block
    x = layers.Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same", name="Conv1")(input_img)
    x = layers.MaxPooling2D((2, 2), name="pool1")(x)
    
    # Second conv block
    x = layers.Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same", name="Conv2")(x)
    x = layers.MaxPooling2D((2, 2), name="pool2")(x)
    
    # We have used two max pool with pool size and strides 2.
    # Hence, downsampled feature maps are 4x smaller.
    # The number of filters in the last layer is 64. 
    # Reshape accordingly before passing the output to the RNN part of the model
    new_shape = ((img_width // 4), (img_height // 4) * 64)
    x = layers.Reshape(target_shape=new_shape, name="reshape")(x)
    x = layers.Dense(64, activation="relu", name="dense1")(x)
    x = layers.Dropout(0.2)(x)
    
    # RNNs
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.25))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.25))(x)
    
    # Output layer
    x = layers.Dense(len(characters) + 1, activation="softmax", name="dense2")(x)
    
    # Add CTC layer for calculating CTC loss at each step
    output = CTCLayer(name="ctc_loss")(labels, x)
    
    # Define the model 
    model = keras.models.Model(
        inputs=[input_img, labels], outputs=output, name="ocr_model_v1"
    )
    # Optimizer
    opt = keras.optimizers.Adam()
    # Compile the model and return
    model.compile(optimizer=opt)
    return model





In [ ]:
# Get the model

model = build_model()
#model.save('/home/pouria/Desktop/dd/')
model.summary()

# Training

In [ ]:
epochs = 100
early_stopping_patience = 10
#model = load_model('/home/pouria/Desktop/dd/')
# Add early stopping

#early_stopping = keras.callbacks.EarlyStopping(
 #monitor="val_loss", patience=early_stopping_patience, restore_best_weights=True
#)


# Train the model
#history = model.fit(
#    train_dataset, 
#    validation_data=validation_dataset,
#    epochs=epochs,
#    callbacks=[early_stopping]
#)
#model.save('/Users/pouriajangjoo/Desktop/tensorflow_macos')

# Inference

In [ ]:
# Get the prediction model by extracting layers till the output layer
model = keras.models.load_model('/Users/pouria/Desktop/osx ')
prediction_model = keras.models.Model(
    model.get_layer(name="image").input, model.get_layer(name="dense2").output
)



#predict the result
prediction_model.summary()

# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][:, : max_length]
    # Iterate over the results and get back the text
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text

# Let's check results on some validation samples
for batch in validation_dataset.take(1):
    batch_images = batch["image"]
    batch_labels = batch["label"]
    
    preds = prediction_model.predict(batch_images)
    pred_texts = decode_batch_predictions(preds)
    
    orig_texts = []
    for label in batch_labels:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        orig_texts.append(label)
        
    _, ax = plt.subplots(4, 4, figsize=(15, 5))
    for i in range(len(pred_texts)):
        img = (batch_images[i, :, :, 0] * 255).numpy().astype(np.uint8)
        img = img.T
        title = f"Prediction: {pred_texts[i]}"
        ax[i // 4, i % 4].imshow(img, cmap="gray")
        ax[i // 4, i % 4].set_title(title)
        ax[i // 4, i % 4].axis("off")  
      
plt.show()

In [ ]:
import os
import time
import threading
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from datetime import datetime,timedelta
from selenium.webdriver.common.keys import Keys
from multiprocessing import Process
from selenium.webdriver.chrome.service import Service
import base64
import pickle
from PIL import Image
from io import BytesIO
import urllib.request
import requests
import time
def saveImage(html,name):
    html = str(html)
    list = html.split('data:image/jpg;base64,')
    list2 = list[1].split('\'')
    data = list2[0]
    im = Image.open(BytesIO(base64.b64decode(data)))
    im.save(name+'.png', 'PNG')


In [ ]:

def getCaptcha():
    test_images = list(map(str, list(Path("./test/").glob("*.png"))))
    test_labels = [img.split(os.path.sep)[-1].split(".png")[0] for img in test_images]
    test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
    test_dataset = (
        test_dataset.map(
            encode_single_sample, num_parallel_calls=tf.data.experimental.AUTOTUNE
        )
        .batch(batch_size)
        .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    )
    for batch in test_dataset.take(1):
        pred_texts = decode_batch_predictions(prediction_model.predict(batch["image"]))
    return str(pred_texts[0])

In [ ]:

#tarikhe ye rooz qabl
kar="https://service2.diplo.de/rktermin/extern/appointment_showDay.do?locationCode=amma&realmId=16&categoryId=44&dateStr=19.09.2021"
urlirani = "https://service2.diplo.de/rktermin/extern/appointment_showDay.do?locationCode=eriw&realmId=351&categoryId=569&dateStr=30.09.2021"
url = "https://service2.diplo.de/rktermin/extern/appointment_showDay.do?locationCode=eriw&realmId=351&categoryId=1513&dateStr=18.09.2021"
#date = "06.04.2021"
x = datetime(2021,9,17,0,29,0)
def open(sec):
    options = Options()
    options.add_argument("--window-size=800,600");
    options.page_load_strategy = 'none'
    #options.add_argument("--disable-javascript")
    #prefs = {"profile.managed_default_content_settings.images": 2}
    #options.add_experimental_option("prefs", prefs)
    #options.add_extension('/Users/pouriajangjoo/Desktop/extension_0_1_0_0.crx')
    browser = webdriver.Chrome(options=options,executable_path='/Users/pouriajangjoo/Desktop/osx/dd/chromedriver')
    #browser=webdriver.Firefox(executable_path='/Users/pouriajangjoo/Desktop/osx/dd/geckodriver')
    browser.get(kar)
    time.sleep(3)
    saveImage(str(browser.page_source), 'test/mmmmmm')
    browser.find_element(By.ID,'appointment_captcha_day_captchaText').send_keys(getCaptcha())
    browser.execute_script("""document.querySelector("input[name='action:appointment_showDay']").click();""")
    while(x > datetime.now()):
        pass 
    browser.get('https://service2.diplo.de/rktermin/extern/appointment_showForm.do?locationCode=amma&realmId=16&categoryId=44&dateStr=23.09.2021&openingPeriodId=61711')
    start = datetime.now()
    print(datetime.now())
    saveImage(str(browser.page_source), 'test/mmmmmm')
    browser.find_element(By.ID,'appointment_newAppointmentForm_captchaText').send_keys(getCaptcha())
    browser.execute_script("""document.getElementById("appointment_newAppointmentForm_lastname").value="keshvari";""")
    browser.execute_script("""document.getElementById("appointment_newAppointmentForm_firstname").value="arash";""")
    browser.execute_script("""document.getElementById("appointment_newAppointmentForm_email").value="@gmail.com";""")
    browser.execute_script("""document.getElementById("appointment_newAppointmentForm_emailrepeat").value="@gmail.com";""")
    browser.execute_script("""document.getElementById("appointment_newAppointmentForm_fields_0__content").value="F49698846";""")
    browser.execute_script("""document.getElementById("appointment_newAppointmentForm_fields_1__content").value="003749308457";""")
     #shomare_telepono_avaz_kon
    browser.execute_script("""document.getElementById("appointment_newAppointmentForm_fields_2__content").value="Studium / studies";""")
    browser.execute_script("""document.getElementById("appointment_newAppointmentForm_fields_3__content").value="Iran";""")
    element=browser.find_element(By.ID,"appointment_newAppointmentForm_appointment_addAppointment")
    javaScript = "document.getElementById('appointment_newAppointmentForm_appointment_addAppointment').click();"
    end = start + timedelta(seconds = sec)
    while(end > datetime.now()):
         pass
    browser.execute_script(javaScript);
    print(datetime.now())
sec = 3.94
t1 = threading.Thread(target=open , args=(sec,))
t1.start()  